In [ ]:
import pandas as pd
import json
from fuzzywuzzy import process
from mlx_lm import load, generate
import re
import functools
import threading
import time
import sys

company_names = {
    "ucb": "UCB.BR",
    "abinbev": "ABI.BR",
    "aedifica": "AED.BR",
    "ageas": "AGS.BR",
    "aperam": "APAM.AS",
    "argenx": "ARGX.BR",
    "belgian national bank": "BNB.BR",
    "cofinimmo": "COFB.BR",
    "elia": "ELI.BR",
    "ishares msci belgium etf": "EWK",
    "groupe bruxelles lambert sa": "GBLB.BR",
    "galapagos nv": "GLPG.AS",
    "kbc bank": "KBC.BR",
    "melexis nv": "MELE.BR",
    "proximus": "PROX.BR",
    "sofina": "SOF.BR",
    "solvay": "SOLB.BR",
    "umicore": "UMI.BR",
    "unified post group": "UPG.BR",
    "warehouses de pauw": "WDP.BR",
    "xior student housing": "XIOR.BR",
}

companies = list(company_names.keys())
# Threshold for acceptable similarity (0-100 scale)
threshold = 80
model, tokenizer = load("mlx-community/Llama-3.2-3B-Instruct-4bit")


def find_best_match(user_input, company_list, threshold):
    # Find the best match with score
    best_match, best_score = process.extractOne(user_input, company_list)

    # Check if the best match meets the threshold
    if best_score >= threshold:
        return best_match
    else:
        return "none"


def read_file_to_string(file_name):
    try:
        with open(file_name, "r", encoding="utf-8") as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return "Error: File not found."
    except IOError:
        return "Error: Could not read the file."


def retrieve_stock_etf_info(companies: list, stock_name: str, threshold: int) -> str:
    stock_name = stock_name.lower()
    result = find_best_match(stock_name, companies, threshold)
    # let's do a fuzzy match with the company names and retrieve the most relevant company name
    res = ""
    if result != "none":
        file_content = read_file_to_string("data/" + company_names[result] + ".txt")
        res = json.dumps({"information we have on stock/etf": file_content})
    else:
        res = json.dumps({"error": "transaction id not found."})
    return res


def extract_json(input):

    json_match = re.search(r"\{.*\}", input)
    if json_match:
        json_string = json_match.group()
        data = json.loads(json_string)
        return data
    else:
        return None


def tool_call(message, model, tokenizer):
    message = [
        {
            "role": "user",
            "content": message,
        }
    ]

    # Define tools available for the model to use:
    tool_message = [
        {
            "name": "retrieve_stock_etf_info",
            "description": "get info about a certain stock or etf given the name of the instrument",
            "parameter_definitions": {
                "stock_name": {
                    "description": "Name of the stock or ETF for which we need information",
                    "type": "str",
                    "required": True,
                }
            },
        },
        {
            "name": "directly_answer",
            "description": "Calls a standard (un-augmented) AI chatbot to generate a response given the conversation history",
            "parameter_definitions": {},
        },
    ]
    formatted_input = tokenizer.apply_chat_template(
        message, tools=tool_message, tokenize=False, add_generation_prompt=True
    )
    response = generate(model, tokenizer, prompt=formatted_input, verbose=False)
    json = extract_json(response)
    if json is None or json["name"] == "directly_answer":
        return None
    else:
        names_to_functions = {
            "retrieve_stock_etf_info": functools.partial(
                retrieve_stock_etf_info, companies=companies, threshold=80
            ),
        }

        # to be called after the tool answer
        function_name = json["name"]
        function_params = json["parameters"]
        function_result = names_to_functions[function_name](**function_params)
        return function_result


def message_call(message, tool_response, model, tokenizer):
    if tool_response is not None:
        text = (
            "answer the following question using the given information. Question : '"
            + message
            + "'. Information : '"
            + tool_response
            + "'."
        )
        messages = [
            {
                "role": "user",
                "content": text,
            }
        ]
    else:
        messages = [
            {
                "role": "user",
                "content": message,
            }
        ]
    formatted_input = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    response = generate(model, tokenizer, prompt=formatted_input, verbose=False)
    return response

/Users/naimsassine/anaconda3/envs/mistral_ai/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
input_txt = "what can you tell me about the ABinBev stock?"
tool_answer = tool_call(input_txt, model, tokenizer)
final_answer = message_call(input_txt, tool_answer, model, tokenizer)

In [4]:
final_answer

'Based on the provided information, here are some key points about the AB Inbev stock:\n\n1. **Current Price**: The current last traded price of AB Inbev stock is €59.74.\n2. **Price Change**: The stock has increased by 0.16 euros since the market opened.\n3. **Price Change Since Close**: The stock has increased by 0.20 euros since the market closed.\n4. **Trading Exchange**: AB Inbev stock is traded on'

In [5]:
# Import necessary modules
# Assuming tool_call and message_call functions are already defined


def chat_mode():
    print("Welcome to the Chat Interface! Type 'exit' to end the session.\n")

    while True:
        # Step 1: Get input from the user
        user_input = input("You: ")

        # Step 2: Exit condition
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        try:
            # Step 3: Process the input using tool_call
            tool_answer = tool_call(user_input, model, tokenizer)

            # Step 4: Generate the final answer using message_call
            final_answer = message_call(user_input, tool_answer, model, tokenizer)

            # Step 5: Display the response to the user
            print(f"Assistant: {final_answer}")

        except Exception as e:
            # Handle unexpected errors gracefully
            print(f"Assistant: Sorry, something went wrong. ({str(e)})")

In [ ]:
# Function to display a spinner
def spinner_animation(message="Processing..."):
    spinner_chars = "|/-\\"
    idx = 0
    while not stop_spinner_event.is_set():
        sys.stdout.write(f"\r{message} {spinner_chars[idx]}")
        sys.stdout.flush()
        idx = (idx + 1) % len(spinner_chars)
        time.sleep(0.1)
    sys.stdout.write("\r" + " " * (len(message) + 4) + "\r")  # Clear the line


def chat_mode():
    print("Welcome to the Chat Interface! Type 'exit' to end the session.\n")

    # Initialize history
    chat_history = []

    while True:
        try:
            # Step 1: Get input from the user
            user_input = input("You: ")

            # Step 2: Exit condition
            if user_input.lower() == "exit":
                print("Goodbye!")
                break

            # Step 3: Combine chat history with the new input
            context = "\n".join(chat_history + [f"You: {user_input}"])

            # Step 4: Start spinner in a separate thread
            global stop_spinner_event
            stop_spinner_event = threading.Event()
            spinner_thread = threading.Thread(target=spinner_animation)
            spinner_thread.start()

            # Step 5: Process the input using tool_call and message_call
            tool_answer = tool_call(context, model, tokenizer)
            final_answer = message_call(context, tool_answer, model, tokenizer)

            # Step 6: Stop the spinner
            stop_spinner_event.set()
            spinner_thread.join()

            # Step 7: Add to chat history
            chat_history.append(f"You: {user_input}")
            chat_history.append(f"Assistant: {final_answer}")

            # Step 8: Display the response to the user
            print(f"Assistant: {final_answer}")

        except Exception as e:
            # Stop the spinner in case of an error
            stop_spinner_event.set()
            spinner_thread.join()
            # Handle unexpected errors gracefully
            print(f"Assistant: Sorry, something went wrong. ({str(e)})")

In [10]:
chat_mode()

Welcome to the Chat Interface! Type 'exit' to end the session.

Assistant: Not much. Just here to help. What's on your mind?
Assistant: Based on the provided information, the current trading price of the UCB stock is €178.0.
Assistant: Based on the provided information, the last trading price of the Belgian National Bank (BNB) stock is €419.0.
Assistant: Based on the provided information, the volume of the UCB stock is 115,187.
Assistant: Based on the provided information, the last trading price of the UCB stock is €178.0. 

The last trading price of the Belgian National Bank (BNB) stock is €419.0.

The volume of the UCB stock is 115,187.

The volume of the Belgian National Bank (BNB) stock is not available in the provided information.
Assistant: Based on the provided information, I can tell you about the two stocks we have knowledge about:

1. UCB stock:
   - Last trading price: €178.0
   - Currency: EUR
   - Stock exchange on which it's traded: BRU
   - Volume: 115,187
   - Turnover: